<a href="https://colab.research.google.com/github/anvy87/AlgoTrading/blob/master/analysingRatiosScraped.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as ur

In [0]:
company = 'TATAMOTORS.NS'
url_is = 'https://finance.yahoo.com/quote/' + company + '/financials?p=' + company
# url_bs = 'https://finance.yahoo.com/quote/' + company + '/balance-sheet?p=' + company
url_cf = 'https://finance.yahoo.com/quote/' + company + '/cash-flow?p=' + company

In [0]:
is_data = ur.urlopen(url_is).read()
soup_is = BeautifulSoup(is_data, 'lxml')

In [0]:
# bs_data = ur.urlopen(url_bs).read()
# soup_bs = BeautifulSoup(bs_data, 'lxml')

In [0]:
cf_data = ur.urlopen(url_cf).read()
soup_cf = BeautifulSoup(cf_data, 'lxml')

In [0]:
ls_is = []
for l in soup_is.find_all('div'):
  ls_is.append(l.string)
ls_is = [e for e in ls_is if e not in('Operating Expenses', 'Non-recurring Events')]

In [0]:
# ls_bs = []
#for l in soup_bs.find_all('div'):
 # ls_bs.append(l.string)
# ls_bs = [e for e in ls_bs if e not in('Operating Expenses', 'Non-recurring Events')]

In [0]:
ls_cf = []
for l in soup_cf.find_all('div'):
  ls_cf.append(l.string)
ls_cf = [e for e in ls_cf if e not in('Operating Expenses', 'Non-recurring Events')]

In [0]:
new_ls_is = list(filter(None,ls_is))
# new_ls_bs = list(filter(None,ls_bs))
new_ls_cf = list(filter(None,ls_cf))

In [0]:
new_ls_is = new_ls_is[12:]
# new_ls_bs = new_ls_bs[12:]
new_ls_cf = new_ls_cf[12:]

In [0]:
is_data = list(zip(*[iter(new_ls_is)]*6))
# bs_data = list(zip(*[iter(new_ls_bs)]*6))
cf_data = list(zip(*[iter(new_ls_cf)]*6))

In [0]:
Income_st = pd.DataFrame(is_data[0:])
# BalS_st = pd.DataFrame(bs_data)
CF_st = pd.DataFrame(cf_data[0:])

In [0]:
Income_st.columns = Income_st.iloc[0]
Income_st = Income_st.iloc[1:,]
Income_st = Income_st.T
Income_st.columns = Income_st.iloc[0]
Income_st.drop(Income_st.index[0],inplace=True)
Income_st.index.name = ''
Income_st.rename(index={'ttm':'12/31/2019'},inplace=True)
Income_st = Income_st[Income_st.columns[:-5]]

In [0]:
#BalS_st.columns = BalS_st.iloc[0]
#BalS_st = BalS_st.iloc[1:,]
#BalS_st = BalS_st.T
#BalS_st.columns = BalS_st.iloc[0]
#BalS_st.drop(BalS_st.index[0],inplace=True)
#BalS_st.index.name = ''
#BalS_st.rename(index={'ttm':'12/31/2019'},inplace=True)
#BalS_st = BalS_st[BalS_st.columns[:-5]] 

In [0]:
CF_st.columns = CF_st.iloc[0]
CF_st = CF_st.iloc[1:,]
CF_st = CF_st.T
CF_st.columns = CF_st.iloc[0]
CF_st.drop(CF_st.index[0],inplace=True)
CF_st.index.name = ''
CF_st.rename(index={'ttm':'12/31/2019'},inplace=True)
CF_st = CF_st[CF_st.columns[:-5]]

In [0]:
Income_st

Annual,Total Revenue,Cost of Revenue,Gross Profit,Research Development,Selling General and Administrative,Total Operating Expenses,Operating Income or Loss,Interest Expense,Total Other Income/Expenses Net,Income Before Tax,Income Tax Expense,Income from Continuing Operations,Net Income,Net Income available to common shareholders
,,,,,,,,,,,,,,
12/31/2019,"32,901,173","21,761,000","11,140,173","42,089,170","3,784,839","13,282,248","-2,142,074","661,733","-921,711","-3,636,695","-219,685","-3,417,010","-3,428,211","-3,428,211"
3/31/2019,"2,993,662,400","1,978,855,800","1,014,806,600","42,245,700","349,813,400","1,217,898,300","-203,091,700","57,586,000","-64,740,000","-317,553,100","-25,425,000","-292,128,100","-293,142,700","-293,142,700"
3/31/2018,"2,882,951,100","1,869,682,900","1,013,268,200","35,318,700","305,980,700","1,129,005,700","-115,737,500","46,365,000","260,720,600","105,740,500","38,058,500","67,682,000","66,660,800","66,660,800"
3/31/2017,"2,656,495,100","1,670,895,400","985,599,700","34,135,700","287,709,200","1,049,555,800","-63,956,100","42,365,700","198,585,500","97,904,400","35,670,000","62,234,400","61,210,500","61,210,500"
3/31/2016,"2,697,422,000","1,623,075,100","1,074,346,900","34,687,700","291,860,900","1,046,461,300","27,885,600","47,912,600","137,225,300","124,384,900","27,512,700","96,872,200","95,883,400","95,883,400"


In [0]:
CF_st

Annual,Net Income,Depreciation & amortization,Deferred income taxes,Stock based compensation,Change in working capital,Accounts receivable,Inventory,Accounts Payable,Other working capital,Other non-cash items,Net cash provided by operating activites,"Investments in property, plant and equipment","Acquisitions, net",Purchases of investments,Sales/Maturities of investments,Other investing activites,Net cash used for investing activites,Debt repayment,Common stock issued,Common stock repurchased,Dividends Paid,Other financing activites,Net cash used privided by (used for) financing activities,Net change in cash
,,,,,,,,,,,,,,,,,,,,,,,,
12/31/2019,"-3,428,211","2,455,318","-219,685",-,"56,789","219,396","380,372","328,018","-881,923","599,240","2,758,485","-3,640,407",-,"-2,634,432","3,823,341","-28,097","-2,305,048","-3,847,760",-,-,"225,000",-,"1,035,847","1,489,284"
3/31/2019,"-293,142,700","230,197,800","-25,425,000",-,"-63,336,000","9,547,000","20,686,400","-26,810,700","-164,127,700","51,200,600","188,907,500","-353,035,200","-170,700","-241,983,800","384,914,100","-2,816,800","-197,110,900","-351,980,200",0,-,0,"-2,655,600","88,303,700","80,100,300"
3/31/2018,"66,660,800","209,818,200","38,058,500",-,"-58,317,600","-43,265,800","-35,604,300","72,565,700","-112,215,000","3,152,500","238,574,200","-350,789,200","-42,100","-481,970,700","547,243,200","-1,783,600","-262,016,100","-299,638,700",0,-,0,"-1,616,300","20,117,100","-3,324,800"
3/31/2017,"61,210,500","182,405,400","35,670,000",-,"33,822,200","-23,677,800","-66,309,400","84,935,100","-3,490,300","36,725,000","303,107,300","-306,597,600","-1,069,500","-447,493,000","354,115,400","5,204,400","-382,727,600","-217,320,900","51,700",-,"-730,000","-1,621,800","62,053,000","-17,567,300"
3/31/2016,"95,883,400","168,074,900","27,512,700",-,"13,332,800","-7,945,600","-57,341,500","40,776,500","63,110,400","40,726,000","374,713,000","-311,602,600",0,"-481,694,000","416,205,100","-1,884,700","-370,501,300","-235,859,200","74,906,700",-,0,"-2,719,600","-37,928,700","-33,717,000"


In [0]:
def convert_to_int(df):
  col = df.columns
  for col in df:
    temp = df[col].to_string().replace(',','').split('\n')
    temp.pop(0)
    df.index = pd.to_datetime(df.index)
    df[col] = [i[10:].strip() for i in temp]
    df[col] = pd.to_numeric(df[col], errors = 'coerce').fillna(0).astype(int)
    return df

In [0]:
ratio = pd.DataFrame()

In [0]:
ratio['effective tax rate'] = Income_st['Income Tax Expense']/Income_st['Income Before Tax']

In [0]:
ratio['net operating margin'] = (Income_st['Income Before Tax']+Income_st['Interest Expense']*(1-ratio['effective tax rate']))/Income_st['Total Revenue']

In [0]:
ratio['net operating asset turnover'] = Income_st['Total Revenue']/((balance_st['Total liabilities and stockholders\' equity']+balance_st['Total liabilities and stockholders\' equity'].shift(1))/2)

In [0]:
ratio['Return on NOA'] = ratio['net operating margin']*ratio['net operating asset turnover']

In [0]:
ratio['Net financial expense'] = Income_st['Interest Expense']*(1-ratio['effective tax rate'])
ratio['Net borrowing cost'] = ratio['Net financial expense']/(balance_st['Total Liabilities']+balance_st['Total Liabilities'].shift(1)/2)
ratio['spread'] = ratio['Return on NOA'] - ratio['Net borrowing cost']

In [0]:
ratio['leverage'] = (balance_st['Total Liabilities']+balance_st['Total Liabilities'].shift(-1)/2)/(balance_st['Total liabilities and stockholders\' equity']+balance_st['Total liabilities and stockholders\' equity'].shift(1))/2

In [0]:
ratio['ROE(Advanced dupont)'] = ratio['Return on NOA'] + ratio['leverage']* ratio['spread']
ratio